In [1]:
import pandas as pd
import numpy as np
import google_analytics_reporting_api as ga
import json as json
import datetime

In [2]:
# Input
Start_Date='2017-10-22'
End_Date=str(datetime.datetime.now().date()-datetime.timedelta(days=1)) # Yesterday
writer_Category="/home/jian/Projects/Tone_Networks/Output/Tone_Network_GA_Category_Data_From_2017-10-22_To_"+End_Date+".xlsx"
writer_Action="/home/jian/Projects/Tone_Networks/Output/Tone_Network_GA_Action_Data_From_2017-10-22_To_"+End_Date+".xlsx"
writer_CateAct="/home/jian/Projects/Tone_Networks/Output/Tone_Network_GA_Category_and_Action_Data_From_2017-10-22_To_"+End_Date+".xlsx"

In [3]:
writer_Category=pd.ExcelWriter(writer_Category, engine='xlsxwriter')
writer_Action=pd.ExcelWriter(writer_Action, engine='xlsxwriter')
writer_CateAct=pd.ExcelWriter(writer_CateAct, engine='xlsxwriter')


analytics = ga.initialize_analyticsreporting('/home/jian/Projects/Tone_Networks/GA/Tone Networks - prod-00ada014c4ba.json')
general_matric=['ga:sessions','ga:users','ga:bounces','ga:bounceRate','ga:pageviews','ga:uniquePageviews','ga:totalEvents','ga:uniqueEvents']

In [7]:
def num_func_trans(df):
    if 'ga:sessions' in df.columns:
        df['ga:sessions']=df['ga:sessions'].astype(int)
    if 'ga:users' in df.columns:
        df['ga:users']=df['ga:users'].astype(int)
    if 'ga:bounces' in df.columns:
        df['ga:bounces']=df['ga:bounces'].astype(int)
    if 'ga:bounceRate' in df.columns:
        df['ga:bounceRate']=round(df['ga:bounceRate'].astype(float)/100,4)
    if 'ga:pageviews' in df.columns:
        df['ga:pageviews']=df['ga:pageviews'].astype(int)
    if 'ga:uniquePageviews' in df.columns:
        df['ga:uniquePageviews']=df['ga:uniquePageviews'].astype(int)
    if 'ga:totalEvents' in df.columns:
        df['ga:totalEvents']=df['ga:totalEvents'].astype(int)
    if 'ga:uniqueEvents' in df.columns:
        df['ga:uniqueEvents']=df['ga:uniqueEvents'].astype(int)
    if 'ga:goalCompletionsAll' in df.columns:
        df['ga:goalCompletionsAll']=df['ga:goalCompletionsAll'].astype(int)
    if 'ga:dateHourMinute' in df.columns:
        df['ga:dateHourMinute']=df['ga:dateHourMinute'].apply(lambda x:x[0:4]+"-"+x[4:6]+"-"+x[6:8]+" "+x[8:10]+":"+x[10:12])
    if 'ga:date' in df.columns:
        df['ga:date']=df['ga:date'].apply(lambda x:x[0:4]+"-"+x[4:6]+"-"+x[6:8])
    
    return df

In [8]:
Event_Mapping=pd.read_excel("/home/jian/Projects/Tone_Networks/event map.xlsx")
Event_Mapping.columns=['eventCategory','eventCategory_New']

In [19]:
# 0 Date
Date_Genral = ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=general_matric, ga_dimensions=['ga:date'])
Date_Genral=num_func_trans(Date_Genral)
Date_Genral=Date_Genral[['ga:date']+general_matric]
Date_Genral.rename(columns=lambda x:x[3:len(x)],inplace=True)


Date_Event=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=['ga:totalEvents'], ga_dimensions=['ga:date','ga:eventCategory','ga:eventAction','ga:eventLabel'])
Date_Event=num_func_trans(Date_Event)
Date_Event_Category=Date_Event[['ga:date','ga:eventCategory','ga:totalEvents']]
Date_Event_Category=Date_Event_Category.groupby(['ga:date','ga:eventCategory'])['ga:totalEvents'].sum().to_frame()
Date_Event_Category.reset_index(inplace=True)
Date_Event_Category.rename(columns=lambda x:x[3:len(x)],inplace=True)
Date_Event_Category['eventCategory']=Date_Event_Category['eventCategory'].apply(lambda x: "EventCategory_"+x)
Date_Event_Category=pd.merge(Date_Event_Category,Event_Mapping,on='eventCategory',how='left')
Date_Event_Category=Date_Event_Category[['date','eventCategory_New','totalEvents']]
Date_Event_Category.columns=['date','eventCategory','totalEvents']
###
Date_Event_Category=Date_Event_Category.groupby(['date','eventCategory'])['totalEvents'].sum().to_frame()
Date_Event_Category.reset_index(inplace=True)
###
Date_Event_Category=Date_Event_Category.pivot(index='date',columns='eventCategory',values='totalEvents')
Date_Event_Category.reset_index(inplace=True)
Date_Event_Category=pd.merge(Date_Genral,Date_Event_Category,on="date",how="outer")
Date_Event_Category=Date_Event_Category.fillna(0)

Date_Event_Action=Date_Event[['ga:date','ga:eventAction','ga:totalEvents']]
Date_Event_Action=Date_Event_Action.groupby(['ga:date','ga:eventAction'])['ga:totalEvents'].sum().to_frame()
Date_Event_Action.reset_index(inplace=True)
Date_Event_Action.rename(columns=lambda x:x[3:len(x)],inplace=True)
Date_Event_Action['eventAction']=Date_Event_Action['eventAction'].apply(lambda x: "EventAction_"+x)
Date_Event_Action=Date_Event_Action.pivot(index='date',columns='eventAction',values='totalEvents')
Date_Event_Action.reset_index(inplace=True)
Date_Event_Action=pd.merge(Date_Genral,Date_Event_Action,on="date",how="outer")
Date_Event_Action=Date_Event_Action.fillna(0)

Date_Event['ga:Cate_Act']=Date_Event['ga:eventCategory']+"_"+Date_Event['ga:eventAction']
Date_Event_Cate_Act=Date_Event[['ga:date','ga:Cate_Act','ga:totalEvents']]
Date_Event_Cate_Act.rename(columns=lambda x:x[3:len(x)],inplace=True)
Date_Event_Cate_Act=pd.pivot_table(data=Date_Event_Cate_Act,index=['date'],columns='Cate_Act',values='totalEvents',aggfunc='sum',fill_value=0)
Date_Event_Cate_Act.reset_index(inplace=True)
Date_Event_Cate_Act=pd.merge(Date_Genral,Date_Event_Cate_Act,on=['date'],how="outer")
Date_Event_Cate_Act=Date_Event_Cate_Act.fillna(0)

Date_Genral.to_excel(writer_Category,"Date_Genral",index=False)
Date_Genral.to_excel(writer_Action,"Date_Genral",index=False)
Date_Genral.to_excel(writer_CateAct,"Date_Genral",index=False)
Date_Event_Category.to_excel(writer_Category,"Date_Event",index=False)
Date_Event_Action.to_excel(writer_Action,"Date_Event",index=False)
Date_Event_Cate_Act.to_excel(writer_CateAct,"Date_Event",index=False)


'\nDate_Event_Category=Date_Event_Category.pivot(index=\'date\',columns=\'eventCategory\',values=\'totalEvents\')\nDate_Event_Category.reset_index(inplace=True)\nDate_Event_Category=pd.merge(Date_Genral,Date_Event_Category,on="date",how="outer")\nDate_Event_Category=Date_Event_Category.fillna(0)\n\nDate_Event_Action=Date_Event[[\'ga:date\',\'ga:eventAction\',\'ga:totalEvents\']]\nDate_Event_Action=Date_Event_Action.groupby([\'ga:date\',\'ga:eventAction\'])[\'ga:totalEvents\'].sum().to_frame()\nDate_Event_Action.reset_index(inplace=True)\nDate_Event_Action.rename(columns=lambda x:x[3:len(x)],inplace=True)\nDate_Event_Action[\'eventAction\']=Date_Event_Action[\'eventAction\'].apply(lambda x: "EventAction_"+x)\nDate_Event_Action=Date_Event_Action.pivot(index=\'date\',columns=\'eventAction\',values=\'totalEvents\')\nDate_Event_Action.reset_index(inplace=True)\nDate_Event_Action=pd.merge(Date_Genral,Date_Event_Action,on="date",how="outer")\nDate_Event_Action=Date_Event_Action.fillna(0)\n\nD

In [20]:
Date_Event_Category

,date,eventCategory,totalEvents
0,2017-11-08,EventCategory_Justuno,3
1,2017-11-21,EventCategory_Justuno,59
2,2017-12-08,EventCategory_Justuno,10
...,...,...,...
675,2018-02-26,EventCategory_Unauthenticated Page - Money Videos,1
676,2018-02-26,EventCategory_Unauthenticated Page - Work - Le...,1
677,2018-02-26,EventCategory_Unauthenticated Page - Work Videos,4


In [10]:
# 1 Time Goal
Time_Goal=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=['ga:goalCompletionsAll']+general_matric, ga_dimensions=['ga:dateHourMinute'])
Time_Goal=num_func_trans(Time_Goal)
Time_Goal=Time_Goal[['ga:dateHourMinute']+general_matric+['ga:goalCompletionsAll']]
Time_Goal.rename(columns=lambda x:x[3:len(x)],inplace=True)

Goal_Completion_All = ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=['ga:goalCompletionsAll'], ga_dimensions=['ga:dateHourMinute','ga:goalCompletionLocation'])
Goal_Completion_All=num_func_trans(Goal_Completion_All)
Goal_Completion_All=Goal_Completion_All[['ga:dateHourMinute','ga:goalCompletionLocation']+['ga:goalCompletionsAll']]
Goal_Completion_All.rename(columns=lambda x:x[3:len(x)],inplace=True)

Goal_Completion_All=Goal_Completion_All.pivot(index='dateHourMinute',columns='goalCompletionLocation',values='goalCompletionsAll')
Goal_Completion_All.reset_index(inplace=True)
Goal_Completion_All.columns=['dateHourMinute','goalCompletionLocation_/interests','goalCompletionLocation_/signup']

Time_Goal=pd.merge(Time_Goal,Goal_Completion_All,on="dateHourMinute",how="outer")
Time_Goal=Time_Goal.fillna(0)
Time_Goal.to_excel(writer_Category,"Time_Goal",index=False)
Time_Goal.to_excel(writer_Action,"Time_Goal",index=False)
Time_Goal.to_excel(writer_CateAct,"Time_Goal",index=False)

In [26]:
# 2 Event and Time
Event_Time=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=['ga:totalEvents'], ga_dimensions=['ga:dateHourMinute','ga:eventCategory','ga:eventAction','ga:eventLabel'])
Event_Time=num_func_trans(Event_Time)
Event_Time=Event_Time[['ga:dateHourMinute','ga:eventCategory','ga:eventAction','ga:eventLabel']+['ga:totalEvents']]
Event_Time.rename(columns=lambda x:x[3:len(x)],inplace=True)

Time_Genral=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=general_matric, ga_dimensions=['ga:dateHourMinute'])
Time_Genral=num_func_trans(Time_Genral)
Time_Genral=Time_Genral[['ga:dateHourMinute']+general_matric]
Time_Genral.rename(columns=lambda x:x[3:len(x)],inplace=True)

Time_Event=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=['ga:totalEvents','ga:uniqueEvents'], ga_dimensions=['ga:dateHourMinute','ga:eventCategory','ga:eventAction','ga:eventLabel'])
Time_Event=num_func_trans(Time_Event)
'''
Time_Event_Unique=Time_Event_Unique.rename(columns=lambda x:x[3:len(x)])

Time_Event_Unique['eventCategory']=Time_Event_Unique['eventCategory'].apply(lambda x: "EventCategory_"+x)
Time_Event_Unique=Time_Event_Unique.groupby(['dateHourMinute','eventCategory','eventAction','eventLabel'])['totalEvents','uniqueEvents'].sum()
Time_Event_Unique.reset_index(inplace=True)
Time_Event_Unique=pd.merge(Time_Event_Unique,Event_Mapping,on='eventCategory',how='left')
del Time_Event_Unique['eventCategory']
Time_Event_Unique=Time_Event_Unique[['dateHourMinute','eventCategory_New','eventAction','eventLabel','totalEvents','uniqueEvents']]
Time_Event_Unique.columns=['dateHourMinute','eventCategory','eventAction','eventLabel','totalEvents','uniqueEvents']

Time_Event_Category=Time_Event[['ga:dateHourMinute','ga:eventCategory','ga:totalEvents']]
Time_Event_Category=Time_Event_Category.groupby(['ga:dateHourMinute','ga:eventCategory'])['ga:totalEvents'].sum().to_frame()
Time_Event_Category.reset_index(inplace=True)
Time_Event_Category.rename(columns=lambda x:x[3:len(x)],inplace=True)
Time_Event_Category['eventCategory']=Time_Event_Category['eventCategory'].apply(lambda x: "EventCategory_"+x)
Time_Event_Category=pd.merge(Time_Event_Category,Event_Mapping,on='eventCategory',how='left')
Time_Event_Category=Time_Event_Category[['dateHourMinute','eventCategory_New','totalEvents']]
Time_Event_Category.columns=['dateHourMinute','eventCategory','totalEvents']
###
Time_Event_Category=Time_Event_Category.groupby(['dateHourMinute','eventCategory'])['totalEvents'].sum().to_frame()
Time_Event_Category.reset_index(inplace=True)
###
Time_Event_Category=Time_Event_Category.pivot(index='dateHourMinute',columns='eventCategory',values='totalEvents')
Time_Event_Category.reset_index(inplace=True)
Time_Event_Category=pd.merge(Time_Genral,Time_Event_Category,on="dateHourMinute",how="outer")
Time_Event_Category=Time_Event_Category.fillna(0)

Time_Event_Action=Time_Event[['ga:dateHourMinute','ga:eventAction','ga:totalEvents']]
Time_Event_Action=Time_Event_Action.groupby(['ga:dateHourMinute','ga:eventAction'])['ga:totalEvents'].sum().to_frame()
Time_Event_Action.reset_index(inplace=True)
Time_Event_Action.rename(columns=lambda x:x[3:len(x)],inplace=True)
Time_Event_Action['eventAction']=Time_Event_Action['eventAction'].apply(lambda x: "EventAction_"+x)
Time_Event_Action=Time_Event_Action.pivot(index='dateHourMinute',columns='eventAction',values='totalEvents')
Time_Event_Action.reset_index(inplace=True)
Time_Event_Action=pd.merge(Time_Genral,Time_Event_Action,on="dateHourMinute",how="outer")
Time_Event_Action=Time_Event_Action.fillna(0)

Time_Event['ga:Cate_Act']=Time_Event['ga:eventCategory']+"_"+Time_Event['ga:eventAction']
Time_Event_Cate_Act=Time_Event[['ga:dateHourMinute','ga:Cate_Act','ga:totalEvents']]
Time_Event_Cate_Act.rename(columns=lambda x:x[3:len(x)],inplace=True)
Time_Event_Cate_Act=pd.pivot_table(data=Time_Event_Cate_Act,index=['dateHourMinute'],columns='Cate_Act',values='totalEvents',aggfunc='sum',fill_value=0)
Time_Event_Cate_Act.reset_index(inplace=True)
Time_Event_Cate_Act=pd.merge(Time_Genral,Time_Event_Cate_Act,on=['dateHourMinute'],how="outer")
Time_Event_Cate_Act=Time_Event_Cate_Act.fillna(0)

del Time_Event['ga:Cate_Act']
Time_Event.to_excel(writer_Category,"EventOnly_Time",index=False)
Time_Event.to_excel(writer_Action,"EventOnly_Time",index=False)
Time_Event.to_excel(writer_CateAct,"EventOnly_Time",index=False)
Time_Event_Category.to_excel(writer_Category,"Time",index=False)
Time_Event_Action.to_excel(writer_Action,"Time",index=False)
Time_Event_Cate_Act.to_excel(writer_CateAct,"Time",index=False)
'''

'\nTime_Event_Unique=Time_Event_Unique.rename(columns=lambda x:x[3:len(x)])\n\nTime_Event_Unique[\'eventCategory\']=Time_Event_Unique[\'eventCategory\'].apply(lambda x: "EventCategory_"+x)\nTime_Event_Unique=Time_Event_Unique.groupby([\'dateHourMinute\',\'eventCategory\',\'eventAction\',\'eventLabel\'])[\'totalEvents\',\'uniqueEvents\'].sum()\nTime_Event_Unique.reset_index(inplace=True)\nTime_Event_Unique=pd.merge(Time_Event_Unique,Event_Mapping,on=\'eventCategory\',how=\'left\')\ndel Time_Event_Unique[\'eventCategory\']\nTime_Event_Unique=Time_Event_Unique[[\'dateHourMinute\',\'eventCategory_New\',\'eventAction\',\'eventLabel\',\'totalEvents\',\'uniqueEvents\']]\nTime_Event_Unique.columns=[\'dateHourMinute\',\'eventCategory\',\'eventAction\',\'eventLabel\',\'totalEvents\',\'uniqueEvents\']\n\nTime_Event_Category=Time_Event[[\'ga:dateHourMinute\',\'ga:eventCategory\',\'ga:totalEvents\']]\nTime_Event_Category=Time_Event_Category.groupby([\'ga:dateHourMinute\',\'ga:eventCategory\'])[\'ga

In [29]:
Time_Event['ga:uniqueEvents'].unique()

array([1, 0, 2])

In [30]:
Time_Event

,ga:dateHourMinute,ga:eventAction,ga:eventCategory,ga:eventLabel,ga:totalEvents,ga:uniqueEvents
0,2017-11-08 23:15,Engagement - Email Signup,Justuno,Adam Test,1,1
1,2017-11-08 23:15,Targeted Overlay,Justuno,Adam Test,2,1
2,2017-11-21 10:12,Targeted Overlay,Justuno,TN: Initial Attempt,1,1
...,...,...,...,...,...,...
5247,2018-02-26 23:47,click,Home Page,Tone Live tab Clicked,1,1
5248,2018-02-26 23:48,click,Profile Page,Default picture changed to: Sky high,1,1
5249,2018-02-26 23:48,click,Profile Page,Default picture changed to: The Cheetah,1,1


In [ ]:
# 3 Event and Geo
Event_Geo=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=['ga:totalEvents'], ga_dimensions=['ga:metro','ga:city','ga:eventCategory','ga:eventAction','ga:eventLabel'])
Event_Geo=num_func_trans(Event_Geo)
Event_Geo=Event_Geo[['ga:metro','ga:city','ga:eventCategory','ga:eventAction','ga:eventLabel']+['ga:totalEvents']]
Event_Geo.rename(columns=lambda x:x[3:len(x)],inplace=True)

Geo_Genral=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=general_matric, ga_dimensions=['ga:metro','ga:city'])
Geo_Genral=num_func_trans(Geo_Genral)
Geo_Genral=Geo_Genral[['ga:metro','ga:city']+general_matric]
Geo_Genral.rename(columns=lambda x:x[3:len(x)],inplace=True)

Geo_Event=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=['ga:totalEvents','ga:uniqueEvents'], ga_dimensions=['ga:metro','ga:city','ga:eventCategory','ga:eventAction','ga:eventLabel'])
Geo_Event=num_func_trans(Geo_Event)
Geo_Event_Category=Geo_Event[['ga:metro','ga:city','ga:eventCategory','ga:totalEvents']]
Geo_Event_Category=Geo_Event_Category.groupby(['ga:metro','ga:city','ga:eventCategory'])['ga:totalEvents'].sum().to_frame()
Geo_Event_Category.reset_index(inplace=True)
Geo_Event_Category.rename(columns=lambda x:x[3:len(x)],inplace=True)
Geo_Event_Category['eventCategory']=Geo_Event_Category['eventCategory'].apply(lambda x: "EventCategory_"+x)
Geo_Event_Category=pd.merge(Geo_Event_Category,Event_Mapping,on='eventCategory',how='left')
Geo_Event_Category=Geo_Event_Category[['metro','city','eventCategory_New','totalEvents']]
Geo_Event_Category.columns=['metro','city','eventCategory','totalEvents']
###
Geo_Event_Category=Geo_Event_Category.groupby(['metro','city','eventCategory'])['totalEvents'].sum().to_frame()
Geo_Event_Category.reset_index(inplace=True)
###
Geo_Event_Category=pd.pivot_table(data=Geo_Event_Category,index=['metro','city'],columns='eventCategory',values='totalEvents',aggfunc='sum',fill_value=0)
Geo_Event_Category.reset_index(inplace=True)
Geo_Event_Category=pd.merge(Geo_Genral,Geo_Event_Category,on=['metro','city'],how="outer")
Geo_Event_Category=Geo_Event_Category.fillna(0)


Geo_Event_Action=Geo_Event[['ga:metro','ga:city','ga:eventAction','ga:totalEvents']]
Geo_Event_Action=Geo_Event_Action.groupby(['ga:metro','ga:city','ga:eventAction'])['ga:totalEvents'].sum().to_frame()
Geo_Event_Action.reset_index(inplace=True)
Geo_Event_Action.rename(columns=lambda x:x[3:len(x)],inplace=True)
Geo_Event_Action['eventAction']=Geo_Event_Action['eventAction'].apply(lambda x: "EventAction_"+x)
Geo_Event_Action=pd.pivot_table(data=Geo_Event_Action,index=['metro','city'],columns='eventAction',values='totalEvents',aggfunc='sum',fill_value=0)
Geo_Event_Action.reset_index(inplace=True)
Geo_Event_Action=pd.merge(Geo_Genral,Geo_Event_Action,on=['metro','city'],how="outer")
Geo_Event_Action=Geo_Event_Action.fillna(0)

Geo_Event['ga:Cate_Act']=Geo_Event['ga:eventCategory']+"_"+Geo_Event['ga:eventAction']
Geo_Event_Cate_Act=Geo_Event[['ga:metro','ga:city','ga:Cate_Act','ga:totalEvents']]
Geo_Event_Cate_Act.rename(columns=lambda x:x[3:len(x)],inplace=True)
Geo_Event_Cate_Act=pd.pivot_table(data=Geo_Event_Cate_Act,index=['metro','city'],columns='Cate_Act',values='totalEvents',aggfunc='sum',fill_value=0)
Geo_Event_Cate_Act.reset_index(inplace=True)
Geo_Event_Cate_Act=pd.merge(Geo_Genral,Geo_Event_Cate_Act,on=['metro','city'],how="outer")
Geo_Event_Cate_Act=Geo_Event_Cate_Act.fillna(0)


del Geo_Event['ga:Cate_Act']
Geo_Event.to_excel(writer_Category,"EventOnly_Geo",index=False)
Geo_Event.to_excel(writer_Action,"EventOnly_Geo",index=False)
Geo_Event.to_excel(writer_CateAct,"EventOnly_Geo",index=False)
Geo_Event_Category.to_excel(writer_Category,"Geo",index=False)
Geo_Event_Action.to_excel(writer_Action,"Geo",index=False)
Geo_Event_Cate_Act.to_excel(writer_CateAct,"Geo",index=False)

In [ ]:
# 4 Time Page
Time_Page=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=general_matric, ga_dimensions=['ga:dateHourMinute','ga:pagePath'])
Time_Page=num_func_trans(Time_Page)
Time_Page=Time_Page[['ga:dateHourMinute','ga:pagePath']+general_matric]
Time_Page.rename(columns=lambda x:x[3:len(x)],inplace=True)

Page_Genral=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=general_matric, ga_dimensions=['ga:dateHourMinute','ga:pagePath'])
Page_Genral=num_func_trans(Page_Genral)
Page_Genral=Page_Genral[['ga:dateHourMinute','ga:pagePath']+general_matric]
Page_Genral.rename(columns=lambda x:x[3:len(x)],inplace=True)

Page_Event=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=['ga:totalEvents','ga:uniqueEvents'], ga_dimensions=['ga:dateHourMinute','ga:pagePath','ga:eventCategory','ga:eventAction','ga:eventLabel'])
Page_Event=num_func_trans(Page_Event)
Page_Event_Category=Page_Event[['ga:dateHourMinute','ga:pagePath','ga:eventCategory','ga:totalEvents']]
Page_Event_Category=Page_Event_Category.groupby(['ga:dateHourMinute','ga:pagePath','ga:eventCategory'])['ga:totalEvents'].sum().to_frame()
Page_Event_Category.reset_index(inplace=True)
Page_Event_Category.rename(columns=lambda x:x[3:len(x)],inplace=True)
Page_Event_Category['eventCategory']=Page_Event_Category['eventCategory'].apply(lambda x: "EventCategory_"+x)
Page_Event_Category=pd.merge(Page_Event_Category,Event_Mapping,on='eventCategory',how='left')
Page_Event_Category=Page_Event_Category[['dateHourMinute','pagePath','eventCategory_New','totalEvents']]
Page_Event_Category.columns=['dateHourMinute','pagePath','eventCategory','totalEvents']
###
Page_Event_Category=Page_Event_Category.groupby(['dateHourMinute','pagePath','eventCategory'])['totalEvents'].sum().to_frame()
Page_Event_Category.reset_index(inplace=True)
###
Page_Event_Category=pd.pivot_table(data=Page_Event_Category,index=['dateHourMinute','pagePath'],columns='eventCategory',values='totalEvents',aggfunc='sum',fill_value=0)
Page_Event_Category.reset_index(inplace=True)
Page_Event_Category=pd.merge(Page_Genral,Page_Event_Category,on=['dateHourMinute','pagePath'],how="outer")
Page_Event_Category=Page_Event_Category.fillna(0)


Page_Event_Action=Page_Event[['ga:dateHourMinute','ga:pagePath','ga:eventAction','ga:totalEvents']]
Page_Event_Action=Page_Event_Action.groupby(['ga:dateHourMinute','ga:pagePath','ga:eventAction'])['ga:totalEvents'].sum().to_frame()
Page_Event_Action.reset_index(inplace=True)
Page_Event_Action.rename(columns=lambda x:x[3:len(x)],inplace=True)
Page_Event_Action['eventAction']=Page_Event_Action['eventAction'].apply(lambda x: "EventAction_"+x)
Page_Event_Action=pd.pivot_table(data=Page_Event_Action,index=['dateHourMinute','pagePath'],columns='eventAction',values='totalEvents',aggfunc='sum',fill_value=0)
Page_Event_Action.reset_index(inplace=True)
Page_Event_Action=pd.merge(Page_Genral,Page_Event_Action,on=['dateHourMinute','pagePath'],how="outer")
Page_Event_Action=Page_Event_Action.fillna(0)

Page_Event['ga:Cate_Act']=Page_Event['ga:eventCategory']+"_"+Page_Event['ga:eventAction']
Page_Event_Cate_Act=Page_Event[['ga:dateHourMinute','ga:pagePath','ga:Cate_Act','ga:totalEvents']]
Page_Event_Cate_Act.rename(columns=lambda x:x[3:len(x)],inplace=True)
Page_Event_Cate_Act=pd.pivot_table(data=Page_Event_Cate_Act,index=['dateHourMinute','pagePath'],columns='Cate_Act',values='totalEvents',aggfunc='sum',fill_value=0)
Page_Event_Cate_Act.reset_index(inplace=True)
Page_Event_Cate_Act=pd.merge(Page_Genral,Page_Event_Cate_Act,on=['dateHourMinute','pagePath'],how="outer")
Page_Event_Cate_Act=Page_Event_Cate_Act.fillna(0)

del Page_Event['ga:Cate_Act']
Page_Event.to_excel(writer_Category,"EventOnly_Time_Page",index=False)
Page_Event.to_excel(writer_Action,"EventOnly_Time_Page",index=False)
Page_Event.to_excel(writer_CateAct,"EventOnly_Time_Page",index=False)
Page_Event_Category.to_excel(writer_Category,"Time_Page",index=False)
Page_Event_Action.to_excel(writer_Action,"Time_Page",index=False)
Page_Event_Cate_Act.to_excel(writer_CateAct,"Time_Page",index=False)

In [ ]:
# 6 Source Medium
Source_Medium=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=general_matric, ga_dimensions=['ga:sourceMedium'])
Source_Medium=num_func_trans(Source_Medium)
Source_Medium=Source_Medium[['ga:sourceMedium']+general_matric]
Source_Medium.rename(columns=lambda x:x[3:len(x)],inplace=True)

Source_Medium_Genral=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=general_matric, ga_dimensions=['ga:sourceMedium'])
Source_Medium_Genral=num_func_trans(Source_Medium_Genral)
Source_Medium_Genral=Source_Medium_Genral[['ga:sourceMedium']+general_matric]
Source_Medium_Genral.rename(columns=lambda x:x[3:len(x)],inplace=True)

Source_Medium_Event=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=['ga:totalEvents','ga:uniqueEvents'], ga_dimensions=['ga:sourceMedium','ga:eventCategory','ga:eventAction','ga:eventLabel'])
Source_Medium_Event=num_func_trans(Source_Medium_Event)
Source_Medium_Event_Category=Source_Medium_Event[['ga:sourceMedium','ga:eventCategory','ga:totalEvents']]
Source_Medium_Event_Category=Source_Medium_Event_Category.groupby(['ga:sourceMedium','ga:eventCategory'])['ga:totalEvents'].sum().to_frame()
Source_Medium_Event_Category.reset_index(inplace=True)
Source_Medium_Event_Category.rename(columns=lambda x:x[3:len(x)],inplace=True)
Source_Medium_Event_Category['eventCategory']=Source_Medium_Event_Category['eventCategory'].apply(lambda x: "EventCategory_"+x)
Source_Medium_Event_Category=pd.merge(Source_Medium_Event_Category,Event_Mapping,on='eventCategory',how='left')
Source_Medium_Event_Category=Source_Medium_Event_Category[['sourceMedium','eventCategory_New','totalEvents']]
Source_Medium_Event_Category.columns=['sourceMedium','eventCategory','totalEvents']
###
Source_Medium_Event_Category=Source_Medium_Event_Category.groupby(['sourceMedium','eventCategory'])['totalEvents'].sum().to_frame()
Source_Medium_Event_Category.reset_index(inplace=True)
###
Source_Medium_Event_Category=pd.pivot_table(data=Source_Medium_Event_Category,index=['sourceMedium'],columns='eventCategory',values='totalEvents',aggfunc='sum',fill_value=0)
Source_Medium_Event_Category.reset_index(inplace=True)
Source_Medium_Event_Category=pd.merge(Source_Medium_Genral,Source_Medium_Event_Category,on=['sourceMedium'],how="outer")
Source_Medium_Event_Category=Source_Medium_Event_Category.fillna(0)

Source_Medium_Event_Action=Source_Medium_Event[['ga:sourceMedium','ga:eventAction','ga:totalEvents']]
Source_Medium_Event_Action=Source_Medium_Event_Action.groupby(['ga:sourceMedium','ga:eventAction'])['ga:totalEvents'].sum().to_frame()
Source_Medium_Event_Action.reset_index(inplace=True)
Source_Medium_Event_Action.rename(columns=lambda x:x[3:len(x)],inplace=True)
Source_Medium_Event_Action['eventAction']=Source_Medium_Event_Action['eventAction'].apply(lambda x: "EventAction_"+x)
Source_Medium_Event_Action=pd.pivot_table(data=Source_Medium_Event_Action,index=['sourceMedium'],columns='eventAction',values='totalEvents',aggfunc='sum',fill_value=0)
Source_Medium_Event_Action.reset_index(inplace=True)
Source_Medium_Event_Action=pd.merge(Source_Medium_Genral,Source_Medium_Event_Action,on=['sourceMedium'],how="outer")
Source_Medium_Event_Action=Source_Medium_Event_Action.fillna(0)

Source_Medium_Event['ga:Cate_Act']=Source_Medium_Event['ga:eventCategory']+"_"+Source_Medium_Event['ga:eventAction']
Source_Medium_Event_Cate_Act=Source_Medium_Event[['ga:sourceMedium','ga:Cate_Act','ga:totalEvents']]
Source_Medium_Event_Cate_Act.rename(columns=lambda x:x[3:len(x)],inplace=True)
Source_Medium_Event_Cate_Act=pd.pivot_table(data=Source_Medium_Event_Cate_Act,index=['sourceMedium'],columns='Cate_Act',values='totalEvents',aggfunc='sum',fill_value=0)
Source_Medium_Event_Cate_Act.reset_index(inplace=True)
Source_Medium_Event_Cate_Act=pd.merge(Source_Medium_Genral,Source_Medium_Event_Cate_Act,on=['sourceMedium'],how="outer")
Source_Medium_Event_Cate_Act=Source_Medium_Event_Cate_Act.fillna(0)

del Source_Medium_Event['ga:Cate_Act']
Source_Medium_Event.to_excel(writer_Category,"EventOnly_SourceMedium",index=False)
Source_Medium_Event.to_excel(writer_Action,"EventOnly_SourceMedium",index=False)
Source_Medium_Event.to_excel(writer_CateAct,"EventOnly_SourceMedium",index=False)
Source_Medium_Event_Category.to_excel(writer_Category,"SourceMedium",index=False)
Source_Medium_Event_Action.to_excel(writer_Action,"SourceMedium",index=False)
Source_Medium_Event_Cate_Act.to_excel(writer_CateAct,"SourceMedium",index=False)

In [38]:
# 7 Gender Age
Gender_Age=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=general_matric, ga_dimensions=['ga:userGender','ga:userAgeBracket'])
Gender_Age=num_func_trans(Gender_Age)
Gender_Age=Gender_Age[['ga:userGender','ga:userAgeBracket']+general_matric]
Gender_Age.rename(columns=lambda x:x[3:len(x)],inplace=True)

Gender_Age_Genral=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=general_matric, ga_dimensions=['ga:userGender','ga:userAgeBracket'])
Gender_Age_Genral=num_func_trans(Gender_Age_Genral)
Gender_Age_Genral=Gender_Age_Genral[['ga:userGender','ga:userAgeBracket']+general_matric]
Gender_Age_Genral.rename(columns=lambda x:x[3:len(x)],inplace=True)

Gender_Age_Event=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=['ga:totalEvents','ga:uniqueEvents'], ga_dimensions=['ga:userGender','ga:userAgeBracket','ga:eventCategory','ga:eventAction','ga:eventLabel'])
Gender_Age_Event=num_func_trans(Gender_Age_Event)
Gender_Age_Event_Category=Gender_Age_Event[['ga:userGender','ga:userAgeBracket','ga:eventCategory','ga:totalEvents']]
Gender_Age_Event_Category=Gender_Age_Event_Category.groupby(['ga:userGender','ga:userAgeBracket','ga:eventCategory'])['ga:totalEvents'].sum().to_frame()
Gender_Age_Event_Category.reset_index(inplace=True)
Gender_Age_Event_Category.rename(columns=lambda x:x[3:len(x)],inplace=True)
Gender_Age_Event_Category['eventCategory']=Gender_Age_Event_Category['eventCategory'].apply(lambda x: "EventCategory_"+x)
Gender_Age_Event_Category=pd.merge(Gender_Age_Event_Category,Event_Mapping,on='eventCategory',how='left')
Gender_Age_Event_Category=Gender_Age_Event_Category[['userGender','userAgeBracket','eventCategory_New','totalEvents']]
Gender_Age_Event_Category.columns=['userGender','userAgeBracket','eventCategory','totalEvents']
###
Gender_Age_Event_Category=Gender_Age_Event_Category.groupby(['userGender','userAgeBracket','eventCategory'])['totalEvents'].sum().to_frame()
Gender_Age_Event_Category.reset_index(inplace=True)
###
Gender_Age_Event_Category=pd.pivot_table(data=Gender_Age_Event_Category,index=['userGender','userAgeBracket'],columns='eventCategory',values='totalEvents',aggfunc='sum',fill_value=0)
Gender_Age_Event_Category.reset_index(inplace=True)
Gender_Age_Event_Category=pd.merge(Gender_Age_Genral,Gender_Age_Event_Category,on=['userGender','userAgeBracket'],how="outer")
Gender_Age_Event_Category=Gender_Age_Event_Category.fillna(0)

Gender_Age_Event_Action=Gender_Age_Event[['ga:userGender','ga:userAgeBracket','ga:eventAction','ga:totalEvents']]
Gender_Age_Event_Action=Gender_Age_Event_Action.groupby(['ga:userGender','ga:userAgeBracket','ga:eventAction'])['ga:totalEvents'].sum().to_frame()
Gender_Age_Event_Action.reset_index(inplace=True)
Gender_Age_Event_Action.rename(columns=lambda x:x[3:len(x)],inplace=True)
Gender_Age_Event_Action['eventAction']=Gender_Age_Event_Action['eventAction'].apply(lambda x: "EventAction_"+x)
Gender_Age_Event_Action=pd.pivot_table(data=Gender_Age_Event_Action,index=['userGender','userAgeBracket'],columns='eventAction',values='totalEvents',aggfunc='sum',fill_value=0)
Gender_Age_Event_Action.reset_index(inplace=True)
Gender_Age_Event_Action=pd.merge(Gender_Age_Genral,Gender_Age_Event_Action,on=['userGender','userAgeBracket'],how="outer")
Gender_Age_Event_Action=Gender_Age_Event_Action.fillna(0)

Gender_Age_Event['ga:Cate_Act']=Gender_Age_Event['ga:eventCategory']+"_"+Gender_Age_Event['ga:eventAction']
Gender_Age_Event_Cate_Act=Gender_Age_Event[['ga:userGender','ga:userAgeBracket','ga:Cate_Act','ga:totalEvents']]
Gender_Age_Event_Cate_Act.rename(columns=lambda x:x[3:len(x)],inplace=True)
Gender_Age_Event_Cate_Act=pd.pivot_table(data=Gender_Age_Event_Cate_Act,index=['userGender','userAgeBracket'],columns='Cate_Act',values='totalEvents',aggfunc='sum',fill_value=0)
Gender_Age_Event_Cate_Act.reset_index(inplace=True)
Gender_Age_Event_Cate_Act=pd.merge(Gender_Age_Genral,Gender_Age_Event_Cate_Act,on=['userGender','userAgeBracket'],how="outer")
Gender_Age_Event_Cate_Act=Gender_Age_Event_Cate_Act.fillna(0)

del Gender_Age_Event['ga:Cate_Act']
Gender_Age_Event.to_excel(writer_Category,"EventOnly_Gender_Age",index=False)
Gender_Age_Event.to_excel(writer_Action,"EventOnly_Gender_Age",index=False)
Gender_Age_Event.to_excel(writer_CateAct,"EventOnly_Gender_Age",index=False)
Gender_Age_Event_Category.to_excel(writer_Category,"Gender_Age",index=False)
Gender_Age_Event_Action.to_excel(writer_Action,"Gender_Age",index=False)
Gender_Age_Event_Cate_Act.to_excel(writer_CateAct,"Gender_Age",index=False)

/usr/local/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [39]:
# 8 In Market Segment
In_Market_Segment=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=general_matric, ga_dimensions=['ga:interestInMarketCategory'])
In_Market_Segment=num_func_trans(In_Market_Segment)
In_Market_Segment=In_Market_Segment[['ga:interestInMarketCategory']+general_matric]
In_Market_Segment.rename(columns=lambda x:x[3:len(x)],inplace=True)

Segment_Genral=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=general_matric, ga_dimensions=['ga:interestInMarketCategory'])
Segment_Genral=num_func_trans(Segment_Genral)
Segment_Genral=Segment_Genral[['ga:interestInMarketCategory']+general_matric]
Segment_Genral.rename(columns=lambda x:x[3:len(x)],inplace=True)


Segment_Event=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=['ga:totalEvents','ga:uniqueEvents'], ga_dimensions=['ga:interestInMarketCategory','ga:eventCategory','ga:eventAction','ga:eventLabel'])
Segment_Event=num_func_trans(Segment_Event)
Segment_Event_Category=Segment_Event[['ga:interestInMarketCategory','ga:eventCategory','ga:totalEvents']]
Segment_Event_Category=Segment_Event_Category.groupby(['ga:interestInMarketCategory','ga:eventCategory'])['ga:totalEvents'].sum().to_frame()
Segment_Event_Category.reset_index(inplace=True)
Segment_Event_Category.rename(columns=lambda x:x[3:len(x)],inplace=True)
Segment_Event_Category['eventCategory']=Segment_Event_Category['eventCategory'].apply(lambda x: "EventCategory_"+x)
Segment_Event_Category=pd.merge(Segment_Event_Category,Event_Mapping,on='eventCategory',how='left')
Segment_Event_Category=Segment_Event_Category[['interestInMarketCategory','eventCategory_New','totalEvents']]
Segment_Event_Category.columns=['interestInMarketCategory','eventCategory','totalEvents']
###
Segment_Event_Category=Segment_Event_Category.groupby(['interestInMarketCategory','eventCategory'])['totalEvents'].sum().to_frame()
Segment_Event_Category.reset_index(inplace=True)
###
Segment_Event_Category=pd.pivot_table(data=Segment_Event_Category,index=['interestInMarketCategory'],columns='eventCategory',values='totalEvents',aggfunc='sum',fill_value=0)
Segment_Event_Category.reset_index(inplace=True)
Segment_Event_Category=pd.merge(Segment_Genral,Segment_Event_Category,on=['interestInMarketCategory'],how="outer")
Segment_Event_Category=Segment_Event_Category.fillna(0)


Segment_Event_Action=Segment_Event[['ga:interestInMarketCategory','ga:eventAction','ga:totalEvents']]
Segment_Event_Action=Segment_Event_Action.groupby(['ga:interestInMarketCategory','ga:eventAction'])['ga:totalEvents'].sum().to_frame()
Segment_Event_Action.reset_index(inplace=True)
Segment_Event_Action.rename(columns=lambda x:x[3:len(x)],inplace=True)
Segment_Event_Action['eventAction']=Segment_Event_Action['eventAction'].apply(lambda x: "EventAction_"+x)
Segment_Event_Action=pd.pivot_table(data=Segment_Event_Action,index=['interestInMarketCategory'],columns='eventAction',values='totalEvents',aggfunc='sum',fill_value=0)
Segment_Event_Action.reset_index(inplace=True)
Segment_Event_Action=pd.merge(Segment_Genral,Segment_Event_Action,on=['interestInMarketCategory'],how="outer")
Segment_Event_Action=Segment_Event_Action.fillna(0)

Segment_Event['ga:Cate_Act']=Segment_Event['ga:eventCategory']+"_"+Segment_Event['ga:eventAction']
Segment_Event_Cate_Act=Segment_Event[['ga:interestInMarketCategory','ga:Cate_Act','ga:totalEvents']]
Segment_Event_Cate_Act.rename(columns=lambda x:x[3:len(x)],inplace=True)
Segment_Event_Cate_Act=pd.pivot_table(data=Segment_Event_Cate_Act,index=['interestInMarketCategory'],columns='Cate_Act',values='totalEvents',aggfunc='sum',fill_value=0)
Segment_Event_Cate_Act.reset_index(inplace=True)
Segment_Event_Cate_Act=pd.merge(Segment_Genral,Segment_Event_Cate_Act,on=['interestInMarketCategory'],how="outer")
Segment_Event_Cate_Act=Segment_Event_Cate_Act.fillna(0)

del Segment_Event['ga:Cate_Act']
Segment_Event.to_excel(writer_Category,"EventOnly_Segment",index=False)
Segment_Event.to_excel(writer_Action,"EventOnly_Segment",index=False)
Segment_Event.to_excel(writer_CateAct,"EventOnly_Segment",index=False)
Segment_Event_Category.to_excel(writer_Category,"Segment",index=False)
Segment_Event_Action.to_excel(writer_Action,"Segment",index=False)
Segment_Event_Cate_Act.to_excel(writer_CateAct,"Segment",index=False)

/usr/local/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [41]:
# 9 Affinity
Affinity=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=general_matric, ga_dimensions=['ga:interestAffinityCategory'])
Affinity=num_func_trans(Affinity)
Affinity=Affinity[['ga:interestAffinityCategory']+general_matric]
Affinity.rename(columns=lambda x:x[3:len(x)],inplace=True)

Affinity_Genral=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=general_matric, ga_dimensions=['ga:interestAffinityCategory'])
Affinity_Genral=num_func_trans(Affinity_Genral)
Affinity_Genral=Affinity_Genral[['ga:interestAffinityCategory']+general_matric]
Affinity_Genral.rename(columns=lambda x:x[3:len(x)],inplace=True)

Affinity_Event=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=['ga:totalEvents','ga:uniqueEvents'], ga_dimensions=['ga:interestAffinityCategory','ga:eventCategory','ga:eventAction','ga:eventLabel'])
Affinity_Event=num_func_trans(Affinity_Event)
Affinity_Event_Category=Affinity_Event[['ga:interestAffinityCategory','ga:eventCategory','ga:totalEvents']]
Affinity_Event_Category=Affinity_Event_Category.groupby(['ga:interestAffinityCategory','ga:eventCategory'])['ga:totalEvents'].sum().to_frame()
Affinity_Event_Category.reset_index(inplace=True)
Affinity_Event_Category.rename(columns=lambda x:x[3:len(x)],inplace=True)
Affinity_Event_Category['eventCategory']=Affinity_Event_Category['eventCategory'].apply(lambda x: "EventCategory_"+x)
Affinity_Event_Category=pd.merge(Affinity_Event_Category,Event_Mapping,on='eventCategory',how='left')
Affinity_Event_Category=Affinity_Event_Category[['interestAffinityCategory','eventCategory_New','totalEvents']]
Affinity_Event_Category.columns=['interestAffinityCategory','eventCategory','totalEvents']

Affinity_Event_Category=pd.pivot_table(data=Affinity_Event_Category,index=['interestAffinityCategory'],columns='eventCategory',values='totalEvents',aggfunc='sum',fill_value=0)
Affinity_Event_Category.reset_index(inplace=True)
Affinity_Event_Category=pd.merge(Affinity_Genral,Affinity_Event_Category,on=['interestAffinityCategory'],how="outer")
Affinity_Event_Category=Affinity_Event_Category.fillna(0)

Affinity_Event_Action=Affinity_Event[['ga:interestAffinityCategory','ga:eventAction','ga:totalEvents']]
Affinity_Event_Action=Affinity_Event_Action.groupby(['ga:interestAffinityCategory','ga:eventAction'])['ga:totalEvents'].sum().to_frame()
Affinity_Event_Action.reset_index(inplace=True)
Affinity_Event_Action.rename(columns=lambda x:x[3:len(x)],inplace=True)
Affinity_Event_Action['eventAction']=Affinity_Event_Action['eventAction'].apply(lambda x: "EventAction_"+x)
###
Affinity_Event_Action=Affinity_Event_Action.groupby(['interestAffinityCategory','eventCategory'])['totalEvents'].sum().to_frame()
Affinity_Event_Action.reset_index(inplace=True)
###
Affinity_Event_Action=pd.pivot_table(data=Affinity_Event_Action,index=['interestAffinityCategory'],columns='eventAction',values='totalEvents',aggfunc='sum',fill_value=0)
Affinity_Event_Action.reset_index(inplace=True)
Affinity_Event_Action=pd.merge(Affinity_Genral,Affinity_Event_Action,on=['interestAffinityCategory'],how="outer")
Affinity_Event_Action=Affinity_Event_Action.fillna(0)

Affinity_Event['ga:Cate_Act']=Affinity_Event['ga:eventCategory']+"_"+Affinity_Event['ga:eventAction']
Affinity_Event_Cate_Act=Affinity_Event[['ga:interestAffinityCategory','ga:Cate_Act','ga:totalEvents']]
Affinity_Event_Cate_Act.rename(columns=lambda x:x[3:len(x)],inplace=True)
Affinity_Event_Cate_Act=pd.pivot_table(data=Affinity_Event_Cate_Act,index=['interestAffinityCategory'],columns='Cate_Act',values='totalEvents',aggfunc='sum',fill_value=0)
Affinity_Event_Cate_Act.reset_index(inplace=True)
Affinity_Event_Cate_Act=pd.merge(Affinity_Genral,Affinity_Event_Cate_Act,on=['interestAffinityCategory'],how="outer")
Affinity_Event_Cate_Act=Affinity_Event_Cate_Act.fillna(0)

del Affinity_Event['ga:Cate_Act']
Affinity_Event.to_excel(writer_Category,"EventOnly_Affinity",index=False)
Affinity_Event.to_excel(writer_Action,"EventOnly_Affinity",index=False)
Affinity_Event.to_excel(writer_CateAct,"EventOnly_Affinity",index=False)
Affinity_Event_Category.to_excel(writer_Category,"Affinity",index=False)
Affinity_Event_Action.to_excel(writer_Action,"Affinity",index=False)
Affinity_Event_Cate_Act.to_excel(writer_CateAct,"Affinity",index=False)

/usr/local/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [42]:
writer_Category.save()
writer_Action.save()
writer_CateAct.save()